# Reproduction of DECAF: Generating Fair Synthetic Data Using Causally-Aware Generative Networks

In this notebook we reproduce the results from the paper.

In [14]:
#from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
import torch

## Loading the Adult dataset

In [2]:
names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country",
    "income",
]
def load_adult():
    """Load the Adult dataset in a pandas dataframe"""

    path = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    names = [
        "age",
        "workclass",
        "fnlwgt",
        "education",
        "education-num",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "capital-gain",
        "capital-loss",
        "hours-per-week",
        "native-country",
        "income",
    ]
    df = pd.read_csv(path, names=names, index_col=False)
    df = df.applymap(lambda x: x.strip() if type(x) is str else x)

    for col in df:
        if df[col].dtype == "object":
            df = df[df[col] != "?"]
    
    return df

In [3]:
# Display examples from the dataset
adult_dataset = load_adult()
adult_dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
def load_train_test_data(df):
    """Get GAN training data from Adult dataset"""
    replace = [
        [
            "Private",
            "Self-emp-not-inc",
            "Self-emp-inc",
            "Federal-gov",
            "Local-gov",
            "State-gov",
            "Without-pay",
            "Never-worked",
        ],
        [
            "Bachelors",
            "Some-college",
            "11th",
            "HS-grad",
            "Prof-school",
            "Assoc-acdm",
            "Assoc-voc",
            "9th",
            "7th-8th",
            "12th",
            "Masters",
            "1st-4th",
            "10th",
            "Doctorate",
            "5th-6th",
            "Preschool",
        ],
        [
            "Married-civ-spouse",
            "Divorced",
            "Never-married",
            "Separated",
            "Widowed",
            "Married-spouse-absent",
            "Married-AF-spouse",
        ],
        [
            "Tech-support",
            "Craft-repair",
            "Other-service",
            "Sales",
            "Exec-managerial",
            "Prof-specialty",
            "Handlers-cleaners",
            "Machine-op-inspct",
            "Adm-clerical",
            "Farming-fishing",
            "Transport-moving",
            "Priv-house-serv",
            "Protective-serv",
            "Armed-Forces",
        ],
        [
            "Wife",
            "Own-child",
            "Husband",
            "Not-in-family",
            "Other-relative",
            "Unmarried",
        ],
        ["White", "Asian-Pac-Islander", "Amer-Indian-Eskimo", "Other", "Black"],
        ["Female", "Male"],
        [
            "United-States",
            "Cambodia",
            "England",
            "Puerto-Rico",
            "Canada",
            "Germany",
            "Outlying-US(Guam-USVI-etc)",
            "India",
            "Japan",
            "Greece",
            "South",
            "China",
            "Cuba",
            "Iran",
            "Honduras",
            "Philippines",
            "Italy",
            "Poland",
            "Jamaica",
            "Vietnam",
            "Mexico",
            "Portugal",
            "Ireland",
            "France",
            "Dominican-Republic",
            "Laos",
            "Ecuador",
            "Taiwan",
            "Haiti",
            "Columbia",
            "Hungary",
            "Guatemala",
            "Nicaragua",
            "Scotland",
            "Thailand",
            "Yugoslavia",
            "El-Salvador",
            "Trinadad&Tobago",
            "Peru",
            "Hong",
            "Holand-Netherlands",
        ],
        [">50K", "<=50K"],
    ]

    for row in replace:
        df = df.replace(row, range(len(row)))

    df = df.values
    X = df[:, :14].astype(np.uint32)
    X = MinMaxScaler().fit_transform(X)
    for row in X:
        if row[9]>0:
            row[9] = 1
        elif row[9]<0:
            row[9] = 0
    y = df[:, 14].astype(np.uint8)

    return train_test_split(X, y, test_size=2000, stratify=y)

In [5]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],
]

In [6]:
def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

In [7]:
# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(adult_dataset, dag)
print(dag_seed)

[[8, 6], [8, 14], [8, 12], [8, 3], [8, 5], [0, 6], [0, 12], [0, 14], [0, 1], [0, 5], [0, 3], [0, 7], [9, 6], [9, 5], [9, 14], [9, 1], [9, 3], [9, 7], [13, 5], [13, 12], [13, 3], [13, 1], [13, 14], [13, 7], [5, 6], [5, 12], [5, 14], [5, 1], [5, 7], [5, 3], [3, 6], [3, 12], [3, 14], [3, 1], [3, 7], [6, 14], [12, 14], [1, 14], [7, 14]]


In [8]:
bias_dict_ftu = create_bias_dict(adult_dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

bias_dict_dp = create_bias_dict(adult_dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship']})
print('Bias dict DP:', bias_dict_dp)

bias_dict_cf = create_bias_dict(adult_dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

Bias dict FTU: {14: [9]}
Bias dict DP: {14: [6, 12, 5, 3, 9, 1, 7]}
Bias dict CF: {14: [5, 9]}


In [9]:
# Get training and testing data
X_train, X_test, y_train, y_test = load_train_test_data(adult_dataset)

pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000
mean,0.293559,0.123127,0.119433,0.224449,0.608129,0.175934,0.364138,0.479036,0.111178,0.674988,0.010786,0.020321,0.407120,0.037902
std,0.180110,0.243890,0.071397,0.228510,0.169811,0.195276,0.229000,0.246059,0.299202,0.468388,0.073319,0.092962,0.122538,0.140839
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.150685,0.000000,0.070592,0.066667,0.533333,0.000000,0.153846,0.400000,0.000000,0.000000,0.000000,0.000000,0.397959,0.000000
50%,0.273973,0.000000,0.111883,0.200000,0.600000,0.166667,0.307692,0.400000,0.000000,1.000000,0.000000,0.000000,0.397959,0.000000
75%,0.410959,0.166667,0.151693,0.200000,0.800000,0.333333,0.538462,0.600000,0.000000,1.000000,0.000000,0.000000,0.448980,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Evaluate on original dataset

In [10]:
# Train an MLP
mlp = MLPClassifier().fit(X_train, y_train)

In [11]:
def eval_model(mlp, x = X_test, y = y_test):
    """Helper function that prints evaluation metrics."""

    y_pred = mlp.predict(x[:, :14])

    prec = precision_score(y, y_pred)
    rec = recall_score(y, y_pred)
    roc = roc_auc_score(y, y_pred)
    acc = accuracy_score(y, y_pred)
    
    return prec, rec, roc, acc

def cal_DP(mlp, X_test):
    """Calculate fairness metric DP"""
    
    X_test_df = pd.DataFrame(X_test, columns = names[:-1]) 
    X_test_0 = X_test_df[X_test_df["sex"] <0.5]
    X_test_1 = X_test_df[X_test_df["sex"] >0.5]
    dp = abs(np.mean(mlp.predict(X_test_0)) - np.mean(mlp.predict(X_test_1)))
    
    return dp
    
    
# def FTU(X_test, X_synth, y_synth):
#     """Calculate fairness metric FTU"""
#     X_synth_0 = np.delete(X_synth,9,1) #Delete column of protected attribute
#     X_test_0 = np.delete(X_test,9,1) #Delete column of protected attribute
    
#     mlp_0 = MLPClassifier().fit(X_synth_0, y_synth)
#     mlp_1 = MLPClassifier().fit(X_synth, y_synth)
    
#     ftu = abs(mlp_0.predict_proba(X_test_0) - mlp_1.predict_proba(X_test))
    
#     print("FTU is: ", np.sum(ftu)/len(X_test))

def cal_FTU(mlp, X_test):
    """Calculate fairness metric DP"""
    
    X_test_df = pd.DataFrame(X_test, columns = names[:-1])
    X_test_0 = X_test_df.assign(sex = 0)
    X_test_1 = X_test_df.assign(sex = 1)

    ftu = abs(np.mean(mlp.predict(X_test_0)) - np.mean(mlp.predict(X_test_1)))

    return ftu

In [12]:
# Evaluate original data
y_pred = mlp.predict(X_test)

print(
    "precision: ", precision_score(y_test, y_pred),
    "recall: ", recall_score(y_test, y_pred),
    "roc_auc: ", roc_auc_score(y_test, y_pred),
)

print(cal_DP(mlp, X_test))
print(cal_FTU(mlp, X_test))
#FTU(X_test = X_test, X_synth = X_test, y_synth = y_test)

precision:  0.8788839568801522 recall:  0.9227696404793608 roc_auc:  0.769617751966588
0.17865761243287315
0.028000000000000025


## DECAF

### Train model

In [15]:
from models.DECAF import DECAF
from data import DataModule

train_data = np.column_stack((X_train, y_train))
dm = DataModule(train_data)

model = DECAF(
    dm.dims[0],
    dag_seed=dag_seed,
    h_dim=200,
    lr=0.5e-3,
    batch_size=64,
    lambda_privacy=0,
    lambda_gp=10,
    d_updates=10,
    alpha=2,
    rho=2,
    weight_decay=1e-2,
    grad_dag_loss=False,
    l1_g=0,
    l1_W=1e-4,
    p_gen=-1,
    use_mask=True,
)

trainer = pl.Trainer(max_epochs=10, logger=False)
trainer.fit(model, dm)


torch.save(model.state_dict(), 'DECAF.pkl')

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | generator     | Generator_causal | 134 K 
1 | discriminator | Discriminator    | 43.6 K
---------------------------------------------------
178 K     Trainable params
225       Non-trainable params
178 K     Total params
0.713     Total estimated model params size (MB)


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

AttributeError: 'str' object has no attribute 'copy'

In [17]:
model.load_state_dict(torch.load('DECAF.pkl'))

<All keys matched successfully>

In [19]:
def generate_synthetic_data(biased_edges={}):
    """Generate synthetic data which is also optionally debiased."""
    X_synth = (
        model.gen_synthetic(
            dm.dataset.x,
            gen_order = model.get_gen_order(),
            biased_edges=biased_edges,
        )
        .detach()
        .numpy()
    )

    return X_synth[:, :14], np.rint(X_synth[:,14]).astype(np.uint8)

### Evaluate DECAF-ND

In [20]:
def train_multi(model, mlp, times =10, y_the ="generated", biased_edges={} ):
    prec_list = []
    rec_list = []
    roc_list = []
    dp_list =[]
    ftu_list=[]

    for i in tqdm(range(times)):
        X_synth, y_synth = generate_synthetic_data(biased_edges)
        #X_synth_train, _, y_synth_train, _ = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)
        
        if y_the == "predicted":
            y_synth = mlp.predict(X_synth)
        random.seed(i)
        mlp_synth = MLPClassifier().fit(X_synth, y_synth)

        prec, rec, roc, acc = eval_model(mlp_synth)
        prec_list.append(prec)
        rec_list.append(rec)
        roc_list.append(roc)
        
        dp =  cal_DP(mlp_synth, X_test=X_synth) 
        ftu = cal_FTU(mlp_synth, X_test=X_synth) 
        dp_list.append(dp)
        ftu_list.append(ftu)
    result={"prec_mean":np.mean(prec_list),"prec_std": np.std(prec_list),
           "rec_mean":np.mean(rec_list),"rec_std": np.std(rec_list),
           "roc_mean":np.mean(roc_list),"roc_std": np.std(roc_list),
           "dp_mean":np.mean(dp_list),"dp_std": np.std(dp_list),
           "ftu_mean":np.mean(ftu_list),"ftu_std": np.std(ftu_list),}

    return result

DECAF_ND_result_pred = train_multi(model, mlp, times =10, y_the ="predicted", biased_edges={} )
DECAF_ND_result_pred

  0%|                                                                | 0/10 [00:00<?, ?it/s]E:\Users\anaconda3\envs\py36\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
 10%|█████▌                                                  | 1/10 [01:19<11:59, 79.96s/it]E:\Users\anaconda3\envs\py36\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
 20%|███████████▏                                            | 2/10 [02:39<10:39, 79.92s/it]E:\Users\anaconda3\envs\py36\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conv

{'prec_mean': 0.8664639939335135,
 'prec_std': 0.0014512173391180603,
 'rec_mean': 0.9322237017310252,
 'rec_std': 0.002396804260985351,
 'roc_mean': 0.7494451841988461,
 'roc_std': 0.0025876743951792776,
 'dp_mean': 0.04024304608968612,
 'dp_std': 0.0027883824738367848,
 'ftu_mean': 0.023695050067466826,
 'ftu_std': 0.002187243769100062}

In [68]:
# Generate synthetic data
X_synth, y_synth = generate_synthetic_data()
print(X_synth)
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)


# Train downstream model and eval
print("test on real data")
ndmlp = MLPClassifier().fit(X_synth_train, y_synth_train)

prec, rec, roc, acc = np.mean([eval_model(ndmlp) for i in range(1)], axis=0)
dp = np.mean([ cal_DP(ndmlp, X_test=X_synth) for i in range(1)])
ftu = np.mean([ cal_FTU(ndmlp, X_test=X_synth) for i in range(1)])
print("prec :",prec) 
print("rec :",rec)
print("roc :",roc) 
print("dp :",dp) 
print("ftu :",ftu) 


print("**************")
print("test on synth data")
# the weird thing they did in original code
#eval_model(ndmlp, X_synth_test, y_synth_test)
prec, rec, roc, acc = np.mean([eval_model(ndmlp, X_synth_test, y_synth_test) for i in range(3)], axis=0)
print("prec :",prec) 
print("rec :",rec)
print("roc :",roc)

[[5.01082063e-01 1.46451075e-05 2.05060229e-01 ... 7.43558980e-04
  2.55349755e-01 6.30534759e-19]
 [3.78006548e-01 1.09433067e-08 1.43484667e-01 ... 1.56329976e-14
  3.65474403e-01 2.04087659e-14]
 [1.05403677e-01 7.49660611e-01 2.53465891e-01 ... 6.37287261e-16
  4.94337171e-01 1.36388510e-22]
 ...
 [4.74419177e-01 2.03077008e-07 3.52555066e-02 ... 1.88798919e-01
  5.99800408e-01 2.05215671e-11]
 [9.05574709e-02 1.13401876e-03 7.38045722e-02 ... 6.81791781e-03
  5.24084389e-01 2.82830459e-21]
 [6.52654320e-02 7.67396437e-03 1.07312292e-01 ... 4.61885124e-04
  2.39931121e-01 4.82060132e-05]]
test on real data
prec : 0.7577184720041863
rec : 0.9640479360852197
roc : 0.5171645302916058
dp : 0.046538705123522095
ftu : 0.02208649953838504
**************
test on synth data
prec : 0.7827676240208877
rec : 0.9765472312703581
roc : 0.5409617876781898


E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [69]:
y_synth = mlp.predict(X_synth)
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)

# Train downstream model and eval
ndmlp = MLPClassifier().fit(X_synth_train, y_synth_train)

print("test on real data")
p_r_r_a = eval_model(ndmlp) 
prec, rec, roc, acc = p_r_r_a

dp = cal_DP(ndmlp, X_test=X_synth) 
ftu = cal_FTU(ndmlp, X_test=X_synth)

print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 
print("acc :", acc) 

print("dp :", dp)
print("ftu :", ftu)

print("**************")
print("test on synth data")
# the weird thing they did in original code
#eval_model(ndmlp, X_synth_test, y_synth_test)

prec, rec, roc, acc = np.mean([eval_model(ndmlp, X_synth_test, y_synth_test) for i in range(3)], axis=0)
print("prec :",prec) 
print("rec :",rec)
print("roc :",roc)

test on real data
prec : 0.8668326073428749
rec : 0.9274300932090546
roc : 0.7488556088535232
acc : 0.8385
dp : 0.10046931846391072
ftu : 0.058554079965911554
**************
test on synth data
prec : 0.988404196576477
rec : 0.9905921416712783
roc : 0.940891925757919


E:\Users\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


### Evaluate DECAF-FTU

In [70]:
# Generate synthetic data
X_synth, y_synth = generate_synthetic_data(biased_edges=bias_dict_ftu)
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)

# Train downstream model and eval
ftumlp = MLPClassifier().fit(X_synth_train, y_synth_train)
prec, rec, roc, acc = eval_model(ftumlp)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 
 

ftu = cal_FTU(ftumlp, X_test=X_synth)
dp = cal_DP(ftumlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)

# the weird thing they did in original code
prec, rec, roc, acc = eval_model(ftumlp, X_synth_test, y_synth_test)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

prec : 0.7669172932330827
rec : 0.9507323568575233
roc : 0.5396232065412115
dp is:  0.0205746510105419
ftu is:  0.023648888573254756
prec : 0.7940379403794038
rec : 0.9593975114603799
roc : 0.5780074238062999


E:\Users\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [71]:
y_synth = mlp.predict(X_synth)# Generate synthetic data
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)

ftumlp = MLPClassifier().fit(X_synth_train, y_synth_train)
prec, rec, roc, acc = eval_model(ftumlp)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 
 

ftu = cal_FTU(ftumlp, X_test=X_synth)
dp = cal_DP(ftumlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)

# the weird thing they did in original code
prec, rec, roc, acc = eval_model(ftumlp, X_synth_test, y_synth_test)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

prec : 0.8650990099009901
rec : 0.9307589880159787
roc : 0.7465039919999572
dp is:  0.0878861454951182
ftu is:  0.04889567502308079
prec : 0.9900607399226946
rec : 0.9939024390243902
roc : 0.9510328521652563


E:\Users\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


### Evaluate DECAF-DP

In [72]:
# Generate synthetic data
X_synth, y_synth = generate_synthetic_data(biased_edges=bias_dict_dp)
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)


# Train downstream model and eval
dpmlp = MLPClassifier().fit(X_synth_train, y_synth_train)
prec, rec, roc, acc = eval_model(dpmlp)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc_mean) 

ftu = cal_FTU(dpmlp, X_test=X_synth)
dp = cal_DP(dpmlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)


# the weird thing they did in original code
prec, rec, roc, acc = eval_model(dpmlp, X_synth_test, y_synth_test)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

prec : 0.7619532044760936
rec : 0.9973368841544608
roc : 0.5127353622211883
dp is:  0.0004659545662266673
ftu is:  0.00031957957531425496
prec : 0.7738869434717359
rec : 0.9993540051679587
roc : 0.49967700258397935


E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [73]:
y_synth = mlp.predict(X_synth)# Generate synthetic data
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)

dpmlp = MLPClassifier().fit(X_synth, y_synth)
prec, rec, roc, acc = eval_model(dpmlp)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

ftu = cal_FTU(dpmlp, X_test=X_synth)
dp = cal_DP(dpmlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)


# the weird thing they did in original code
prec, rec, roc, acc = eval_model(dpmlp, X_synth_test, y_synth_test)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

prec : 0.8670377241805813
rec : 0.933422103861518
roc : 0.7508475981154978
dp is:  0.09420044588136312
ftu is:  0.04719125062140472
prec : 0.9928335170893055
rec : 0.9977839335180055
roc : 0.9655586334256696


E:\Users\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


### Evaluate DECAF-CF

In [74]:
# Generate synthetic data
X_synth, y_synth = generate_synthetic_data(biased_edges=bias_dict_cf)
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)


# Train downstream model and eval
cfmlp = MLPClassifier().fit(X_synth_train, y_synth_train)
eval_model(cfmlp)

prec, rec, roc, acc = eval_model(cfmlp)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

ftu = cal_FTU(cfmlp, X_test=X_synth)
dp = cal_DP(cfmlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)

# the weird thing they did in original code
prec, rec, roc, acc = eval_model(cfmlp, X_synth_test, y_synth_test)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

prec : 0.7654004106776181
rec : 0.992676431424767
roc : 0.5375028743469217
dp is:  0.0005579566286761928
ftu is:  0.003408848803352016
prec : 0.7662796567390207
rec : 0.9928057553956835
roc : 0.5048954466999649


E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [75]:
y_synth = cfmlp.predict(X_synth)# Generate synthetic data
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)

cfmlp = MLPClassifier().fit(X_synth_train, y_synth_train)
prec, rec, roc, acc = eval_model(dpmlp)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

ftu = cal_FTU(cfmlp, X_test=X_synth)
dp = cal_DP(cfmlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)


# the weird thing they did in original code
prec, rec, roc, acc = eval_model(dpmlp, X_synth_test, y_synth_test)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

prec : 0.8670377241805813
rec : 0.933422103861518
roc : 0.7508475981154978
dp is:  0.0006908918848490542
ftu is:  0.0038349549037710595
prec : 0.9944382647385984
rec : 0.9021190716448032
roc : 0.6732817580446238


E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


Fairness Calcs

In [62]:
ftu = cal_FTU(cfmlp, X_test=X_synth)
dp = cal_DP(cfmlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)

dp is:  0.004991803404312156
ftu is:  0.004793693629713713


E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
E:\Users\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
